## Topological Sort
### Intuition
Topological sort: Arrange set of nodes with directed edges in linear order such that for every directed edge `(u, v)`, `u` appears before `v`
1. Identify nodes with indegree = 0, place them at beginning of ordering
2. Decrement indegree for each node that depends on the node with indegree = 0, if the new indegree = 0, add them to the array of nodes to be processed
3. Continue until all nodes placed in final order
4. If number of visited noes is less than total nodes, indicates cycle in graph

For this problem, there is variation of topological sort. Directed edges are described by `beforeItems`, but additional requirement that nodes with same group must be next to each other. 

During sorting process, if one node belongs to group `i`, implies that we also have dependencies with all nodes in group `i`.

During topological sorting, gather all nodes belonging to same group and ensure relative order based on dependencies. Do this by using two levels of sorting.
- Sort groups based on group dependencies
- Sort items within each group according to item dependencies

Consider items not belonging to any group as separate groups of a single item.

### Algorithm
1. Initialize `group_id` as `m`. Iterate over each item `i`, if it doesn't belong to a group, assign unique group id `group_id` and increment `group_id` by 1.
2. Create adjacency map `item_graph` for all items and `group_graph` for all groups. Also create indegree list `item_indegree` and `group_indegree`.
3. Traverse `before_items`, add dependency pairs `(prev, cur)` into `item_graph`, increment indegree `cur` by 1. If items belong to different groups `group[prev]` and `group[cur]`, add group dependency to `group_graph` and increment indegree of `group[cur]` by 1
4. Perfrom topological sort of items according to `item_graph` and `item_indegree`. If there is cycle, task is impossible.
5. Perform topological sort of groups based on `group_graph` and `group_indegree`. If there is cycle, task is impossible.
6. Create empty list `answer` to store final order and hash map `ordered_groups` to store sorted items within each group.
7. Iterate over sorted items, for each item `i`, add it to `ordered_groups` while preserving original order within group: `ordered_groups[group[i]].append(i)`
8. Traverse sorted groups, for each group `group_idx`, add all items items `order_groups[group_index]` to `answer` while maintaining original order

In [3]:
from typing import List
from collections import defaultdict

class Solution:
    def sort_items(self, n: int, m: int, group: List[int], before_items: List[List[int]]) -> List[int]:
        group_id = m
        for i in range(n):
            if group[i] == -1:
                group[i] = group_id
                group_id += 1
        
        item_graph = [[] for _ in range(n)]
        item_indegrees = [0] * n
        group_graph = [[] for _ in range(group_id)]
        group_indegrees = [0] * group_id

        for item, dependencies in enumerate(before_items):
            for dependency in dependencies:
                item_graph[dependency].append(item)
                item_indegrees[item] += 1
                if group[item] != group[dependency]:
                    group_graph[group[dependency]].append(group[item])
                    group_indegrees[group[item]] += 1
        
        def topological_sort(graph, indegrees):
            order = []
            stack = [node for node in range(len(graph)) if indegrees[node] == 0]
            while stack:
                cur = stack.pop()
                order.append(cur)
                for neighbor in graph[cur]:
                    indegrees[neighbor] -= 1
                    if indegrees[neighbor] == 0:
                        stack.append(neighbor)
            return order if len(order) == len(graph) else []
        
        item_order = topological_sort(item_graph, item_indegrees)
        group_order = topological_sort(group_graph, group_indegrees)

        if not item_order or not group_order:
            return []

        ordered_groups = defaultdict(list)
        for item in item_order:
            ordered_groups[group[item]].append(item)

        answer = []
        for group_id in group_order:
            answer += ordered_groups[group_id]
        return answer